In [ ]:
import pandas as pd
import os
import re
from tqdm import tqdm

records = pd.read_excel(r"4.xlsx", header=[0])

path = "Data_file/ismt_yices2"
folder_list = os.listdir(path)
for filefolder in folder_list:
    file_path = path + "/" + filefolder
    file_list = os.listdir(file_path)
    solved = {'AProVE': 0, 'calypto': 0, 'Dartagnan': 0, 'LassoRanker': 0, 'leipzig': 0, 'mcm': 0, 'CInteger': 0, 'ITS': 0, 'SAT14': 0, 'total': 0}
    times = {'AProVE': 0, 'calypto': 0, 'Dartagnan': 0, 'LassoRanker': 0, 'leipzig': 0, 'mcm': 0, 'CInteger': 0, 'ITS': 0, 'SAT14': 0, 'total': 0}
    if filefolder == 'smtrat' or filefolder == 'mathsat':
        for filename in tqdm(file_list, desc = filefolder):
            with open(os.path.join(file_path, filename)) as f:
                while f.readline():
                    result = f.readline()
                    if result[:2] == '..':
                        info = result
                        result = 'none'
                    elif result[:2] == 'ER':
                        info = f.readline()
                        while info[:2] != '..':
                            info = f.readline()
                        result = 'ERROR'
                    else:
                        if result[:2] == './' or result[:2] == 'cv' or result[:2] == 'ca':
                            result = 'timeout'
                        elif result[:2] == 'tc':
                            result = 'leak'
                        info = f.readline()
                        if info[:2] == '(e':
                            break
                        while info[:2] == './' or info[:2] == 'tc' or info[:2] == 'cv' or info[:2] == 'ca':
                            info = f.readline()
                        if info[:2] != '..':
                            result = info
                            info = f.readline()
                    info = info.split('/')
                    time = int(info[-1].split(' : ')[1][:-5])
                    info[-1] = info[-1].split(' : ')[0]
                    if result[:3] == 'sat':
                        solved['total'] += 1
                        times['total'] += time
                        if info[4] == '20170427-VeryMax':
                            if(len(info) == 5):
                                solved['SAT14'] += 1
                                times['SAT14'] += time
                            else:
                                solved[info[5]] += 1
                                times[info[5]] += time
                        elif info[4] == 'UltimateAutomizer' or info[4] == 'UltimateLassoRanker':
                            solved['LassoRanker'] += 1
                            times['LassoRanker'] += time
                        elif info[4] == '20210219-Dartagnan':
                            solved['Dartagnan'] += 1
                            times['Dartagnan'] += time
                        elif info[4] != '2019-ezsmt':
                            solved[info[4]] += 1
                            times[info[4]] += time
    else:
        for filename in tqdm(file_list, desc = filefolder):
        # for filename in file_list:
            with open(os.path.join(file_path, filename)) as f:
                flag = False
                while f.readline():
                    result = f.readline()
                    if result[:2] == '..':
                        info = result
                        result = 'none'
                    else:
                        info = f.readline()
                        cnt = 0
                        while info[:3] != '../':
                            if info[:3] == 'sat':
                                result = 'sat'
                            info = f.readline()
                            cnt += 1
                            if cnt > 10000:
                                flag = True
                                break
                    if flag:
                        break
                    # total += 1
                    info = info.split('/')                    
                    # print(filename, result)
                    if result[:3] == 'sat':
                        time = int(info[-1].split(' : ')[1][:-5])
                        solved['total'] += 1
                        times['total'] += time
                        if info[3] == '20170427-VeryMax':
                            if(len(info) == 5):
                                solved['SAT14'] += 1
                                times['SAT14'] += time
                            else:
                                solved[info[4]] += 1
                                times[info[4]] += time
                        elif info[3] == 'UltimateAutomizer' or info[3] == 'UltimateLassoRanker':
                            solved['LassoRanker'] += 1
                            times['LassoRanker'] += time
                        elif info[3] == '20210219-Dartagnan':
                            solved['Dartagnan'] += 1
                            times['Dartagnan'] += time
                        else:
                            solved[info[3]] += 1
                            times[info[3]] += time
    records.loc[records['name'] == filefolder + '_solved', ['AProVE', 'calypto', 'Dartagnan', 'LassoRanker', 'leipzig', 'mcm', 'CInteger', 'ITS', 'SAT14', 'total']] = [solved['AProVE'], solved['calypto'], solved['Dartagnan'], solved['LassoRanker'], solved['leipzig'], solved['mcm'], solved['CInteger'], solved['ITS'], solved['SAT14'], solved['total']]
    records.loc[records['name'] == filefolder + '_time', ['AProVE', 'calypto', 'Dartagnan', 'LassoRanker', 'leipzig', 'mcm', 'CInteger', 'ITS', 'SAT14', 'total']] = [times['AProVE'], times['calypto'], times['Dartagnan'], times['LassoRanker'], times['leipzig'], times['mcm'], times['CInteger'], times['ITS'], times['SAT14'], times['total']]
    # print(total)
print("Waiting...")
records.to_excel('Result/4_1.xlsx')
print("Done")    